In [26]:
from time import sleep
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup


# driver = webdriver.Chrome(ChromeDriverManager().install())


url = "https://www.hltv.org/stats"
players = []
teams = []
teamRanks = []

fields1 = ['Player', 'Team', 'Maps', 'K-D_Diff', 'K/D', 'Rating(1.0)'] 
fields2 = ['Teams' ,'Maps', 'Won-Lost', 'Pistol_Win_%', 'Round_2_Conversion', 'Round_2_Break'] 
fields3 = ['Team', 'Maps', 'K-D_Diff', 'K/D', 'Rating(2.0)']

    
def writeToCSV(file, header, data):
    filename = file

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(header) 

        # writing the data rows 
        csvwriter.writerows(data)
        
        
def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")

def get_parsed_page2(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://ggscore.com/en/csgo",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")

def driving():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.hltv.org/events/5603/blast-premier-spring-showdown-2021')
    element = driver.find_element_by_class_name('team-name')
    print(element)    
        
def playerPerformanceByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    
    dateConversion = url + "/players?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower() + "&rankingFilter=Top50";
    print(dateConversion)
    soup = get_parsed_page(dateConversion)
    statTable = soup.tbody;
    for tr in statTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        players.append(newArray)
    writeToCSV("data.csv", fields1, players)
   

def pistolRoundByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank if you want 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    teamSide = input("Enter team ('TERRORIST or COUNTER_TERRORIST'):")
    pistol = url + "/teams/pistols?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower()+"&side=" + teamSide.upper() + "&rankingFilter=Top50"; 
    print(pistol)
    soup = get_parsed_page(pistol)
    pistolTable = soup.tbody;

    for tr in pistolTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        teams.append(newArray)
    writeToCSV("data2.csv", fields2, teams)

def teamRank():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank if you want 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    teamURL = url + "/teams?startDate="+startDate+"&endDate=" + endDate + "&maps=de_"+ mapName.lower() + "&rankingFilter=Top50"; 
    print(teamURL)
    soup = get_parsed_page(teamURL)
    teamRankTable = soup.tbody;
    
    for tr in teamRankTable.find_all("tr"):
        newArray = []
        for td in tr.find_all("td"):
            newArray.append(td.get_text())
        teamRanks.append(newArray)
    writeToCSV("data3.csv", fields3, teamRanks)
    
    
    
# Calling functions and creating our csv files dynamically    
playerPerformanceByDate()
pistolRoundByDate()
teamRank()
print('-----------------------------------------------------------------------------------------------------------')
print('RUN THE 3 CELLS BELOW TO SEE THE "PLAYER DATAFRAME" AND "TEAM PISTOL DATAFRAME" AND "TEAM RANKS DATAFRAME"')
print('-----------------------------------------------------------------------------------------------------------')

Start date (yyyy-mm-dd, leave blank for 'All-time' results): 
End date (yyyy-mm-dd, leave blank for 'All-time' results): all
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: 
https://www.hltv.org/stats/players?startDate=&endDate=all&maps=de_&rankingFilter=Top50
Start date (yyyy-mm-dd, leave blank for 'All-time' results): 
End date (yyyy-mm-dd, leave blank if you want 'All-time' results): 
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: nuke
Enter team ('TERRORIST or COUNTER_TERRORIST'):terrorist
https://www.hltv.org/stats/teams/pistols?startDate=&endDate=&maps=de_nuke&side=TERRORIST&rankingFilter=Top50
Start date (yyyy-mm-dd, leave blank for 'All-time' results): 
End date (yyyy-mm-dd, leave blank if you want 'All-time' results): 
Map Selection. Choose one from the following: all, vertigo, dust2, mira

In [27]:
#Player Performance dataframe
print("-------------------------------")
print('Player Dataframe')
print("-------------------------------")
player_df = pd.read_csv('data.csv')
player_df = player_df.dropna(axis='columns', how ='all')
player_df.head(20)


-------------------------------
Player Dataframe
-------------------------------


,Player,Maps,K-D_Diff,K/D,Rating(1.0)
0,s1mple,984,5950,1.37,1.25
1,ZywOo,585,3372,1.36,1.25
2,device,1205,4995,1.26,1.16
3,NiKo,1178,3846,1.19,1.15
4,coldzera,1182,4647,1.24,1.15
5,XANTARES,1035,2840,1.15,1.13
6,oskar,975,2991,1.18,1.12
7,Jame,793,3217,1.27,1.12
8,woxic,684,2055,1.18,1.12
9,xsepower,671,2511,1.23,1.11


In [28]:
# Team Pistol dataframe
print("-------------------------------")
print('Team Pistol Dataframe')
print("-------------------------------")
team_pistol_df = pd.read_csv('data2.csv')
team_pistol_df.head(20)

-------------------------------
Team Pistol Dataframe
-------------------------------


,Teams,Maps,Won-Lost,Pistol_Win_%,Round_2_Conversion,Round_2_Break
0,Astralis,155,76 - 79,49.0%,82.9%,27.8%
1,Heroic,189,92 - 97,48.7%,75.0%,14.4%
2,Dignitas,87,45 - 42,51.7%,80.0%,26.2%
3,Sprout,121,58 - 63,47.9%,65.5%,11.1%
4,G2,137,67 - 70,48.9%,77.3%,10.0%
5,mousesports,136,65 - 71,47.8%,76.2%,22.5%
6,Tricked,109,50 - 59,45.9%,70.0%,6.8%
7,Natus Vincere,131,61 - 70,46.6%,70.5%,12.9%
8,Vitality,105,52 - 53,49.5%,61.5%,22.6%
9,forZe,115,56 - 59,48.7%,63.6%,18.6%


In [29]:
# Team Ranks dataframe
print("-------------------------------")
print('Team Overall Rank Dataframe')
print("-------------------------------")
team_rank_df = pd.read_csv('data3.csv')
team_rank_df.head(20)

-------------------------------
Team Overall Rank Dataframe
-------------------------------


,Team,Maps,K-D_Diff,K/D,Rating(2.0)
0,Astralis,155,1675,1.14,1.10
1,North,107,174,1.02,1.08
2,Heroic,189,1237,1.07,1.08
3,G2,137,514,1.04,1.07
4,Sprout,121,506,1.05,1.07
5,Liquid,123,216,1.02,1.07
6,Vitality,105,331,1.04,1.07
7,mousesports,137,567,1.05,1.07
8,Tricked,109,450,1.05,1.07
9,Dignitas,87,462,1.06,1.06


In [21]:
# Get the names of the players in each team depending what team the user chooses.
team_and_player_ratings = 'https://ggscore.com/en/csgo/team/';
teamName = input("Choose a team to view the players on that team: ")
url = team_and_player_ratings + teamName
soup = get_parsed_page2(url);
teamTable = soup.find("div", "teamP")
players = teamTable.find("div", "row")
for name in players.find_all(attrs={"class": "pllink"}):
    print(name.text)

Choose a team to view the players on that team: fiend
mar
REDSTAR
v1c7oR
bubble
dream3r


In [22]:
 player_rating = 'https://ggscore.com/en/csgo/player/'
playerName = input('Enter a player\'s name: ')
url = player_rating + playerName
soup = get_parsed_page2(url)
playerTable = soup.find_all("span", class_="counter")
print(playerName + "'s' Form level is " + playerTable[0].text + " with a Strength level of " + playerTable[1].text) 

Enter a player's name: dream3r
dream3r's' Form level is 108% with a Strength level of 104%


In [33]:
soup = get_parsed_page('https://www.hltv.org/events/5728/dreamhack-masters-spring-2021%27')
teams_in_tournament = soup.find_all("div", "text")

print("---------------------------------------------------------------------------------------------------------")
print("These teams will be in the upcoming tournament known as DreamHack Master Spring 2021 (Apr 29th - May 9th)")
print("---------------------------------------------------------------------------------------------------------")
for team in teams_in_tournament:
    print("-" + team.text)
    


---------------------------------------------------------------------------------------------------------
These teams will be in the upcoming tournament known as DreamHack Master Spring 2021 (Apr 29th - May 9th)
---------------------------------------------------------------------------------------------------------
-Gambit
-Astralis
-Heroic
-Natus Vincere
-Virtus.pro
-FURIA
-G2
-Complexity
-Spirit
-BIG
-Vitality
-mousesports
-FaZe
-Extra Salt
-paiN
-Fiend


In [101]:
g2 = 'https://www.hltv.org/stats/teams/maps/6667/faze'
soup = get_parsed_page(g2)
two_grid = soup.find("div", "two-grid")
map_pool = two_grid.find_all(attrs={"class": "map-pool"})
stats_for_maps = two_grid.find_all("div",'stats-rows standard-box')
maps = []
stuff = []
for map_name in map_pool:
    maps.append(map_name.text)

for map_stat in stats_for_maps:
    stuff.append(map_stat.text)

for i in range(len(maps)):
    print("------------------------------")
    print(maps[i])
    print("------------------------------")
    print(stuff[i])
    print("\n")


------------------------------


Mirage


------------------------------

Wins / draws / losses141 / 3 / 96
Win rate58.8%
Total rounds6352
Round win-% after getting first kill75.4%
Round win-% after receiving first death29.6%



------------------------------


Inferno


------------------------------

Wins / draws / losses97 / 1 / 71
Win rate57.4%
Total rounds4440
Round win-% after getting first kill76.0%
Round win-% after receiving first death29.6%



------------------------------


Train


------------------------------

Wins / draws / losses84 / 1 / 71
Win rate53.8%
Total rounds4048
Round win-% after getting first kill75.2%
Round win-% after receiving first death28.2%



------------------------------


Overpass


------------------------------

Wins / draws / losses82 / 0 / 57
Win rate59.0%
Total rounds3627
Round win-% after getting first kill75.5%
Round win-% after receiving first death29.3%



------------------------------


Dust2


------------------------------

Wins / draws